In [1]:
import os
from llama_index.llms import AzureOpenAI
from llama_index.embeddings import AzureOpenAIEmbedding
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
import logging
import sys
from dotenv import load_dotenv
load_dotenv('/Users/jeana/.env')

logging.basicConfig(
    stream=sys.stdout, level=logging.WARNING
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
api_key = os.environ['OPENAI_API_KEY']
azure_endpoint = os.environ['OPENAI_DEPLOYMENT_ENDPOINT']
api_version = os.environ['OPENAI_DEPLOYMENT_VERSION']

llm = AzureOpenAI(
    model= os.environ['OPENAI_MODEL_NAME'],
    deployment_name= os.environ['OPENAI_DEPLOYMENT_NAME'],
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model=os.environ['OPENAI_EMBEDDING_MODEL_NAME'],
    deployment_name=os.environ['OPENAI_EMBEDDING_DEPLOYMENT_NAME'],
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [3]:
from llama_index import set_global_service_context

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)

set_global_service_context(service_context)

In [5]:
documents = SimpleDirectoryReader(
    input_files=[r"/Users/jeana/Retrieval-Augmented-Generation/LlamaIndex/paul_graham_essay.txt"] #or just indicate the fullpath of the folder containing the data
).load_data()
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

INFO:httpx:HTTP Request: POST https://riskpocai-jpe.openai.azure.com//openai/deployments/AZAIEMBTXTRisk-EDU/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://riskpocai-jpe.openai.azure.com//openai/deployments/AZAIEMBTXTRisk-EDU/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://riskpocai-jpe.openai.azure.com//openai/deployments/AZAIEMBTXTRisk-EDU/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://riskpocai-jpe.openai.azure.com//openai/deployments/AZAIEMBTXTRisk-EDU/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://riskpocai-jpe.openai.azure.com//openai/deployments/AZAIEMBTXTRisk-EDU/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://riskpocai-jpe.openai.azure.com//openai/deployments/AZAIEMBTXTRisk-EDU/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"


In [6]:
query = "What is most interesting about this essay?"
query_engine = index.as_query_engine()
answer = query_engine.query(query)

print(answer.get_formatted_sources())
print("query was:", query)
print("answer was:", answer)

INFO:httpx:HTTP Request: POST https://riskpocai-jpe.openai.azure.com//openai/deployments/AZAIEMBTXTRisk-EDU/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://riskpocai-jpe.openai.azure.com//openai/deployments/AZAIEMBTXTRisk-EDU/embeddings?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://riskpocai-jpe.openai.azure.com//openai/deployments/AZAIPOCRisk-EDU/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://riskpocai-jpe.openai.azure.com//openai/deployments/AZAIPOCRisk-EDU/chat/completions?api-version=2023-07-01-preview "HTTP/1.1 200 OK"
> Source (Doc id: a5a1bdbc-e896-4acb-a37e-12ab851f4a45): One of the most conspicuous patterns I've noticed in my life is how well it has worked, for me at...

> Source (Doc id: 83cbc069-3e87-4f0a-b82d-c57f7c712766): I remember taking the boys to the coast on a sunny day in 2015 and figuring out how to deal with ...
query was: What is most int